In [36]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import norm
import time

Shows movie dataset

In [37]:
movie_dataset = "Dataset/Netflix_Dataset_Movie.csv"
movie_df = pd.read_csv(movie_dataset)
movie_df.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


Shows user dataset

In [38]:
user_dataset = "Dataset/Netflix_Dataset_Rating.csv"
user_df = pd.read_csv(user_dataset)
user_df.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


The below calculation shows how many movies are rated by no users in our dataset. Because these movies have 0 ratings for all users, we will exclude them from our data. Since there are 16,420 movies that are unrated, our user-movie matrix will have signicantly fewer columns than the total number of movies in the dataset.

Sparsity Reduction: Movie recommendation datasets are typically sparse, meaning most movies are not rated by most users. Removing movies that have zero ratings can significantly reduce the sparsity of our matrix, making matrix factorization techniques more effective and computationally feasible.

In [39]:
movies_in_movie_df = set(movie_df['Movie_ID'])
movies_in_user_df = set(user_df['Movie_ID'])
unrated_movies = movies_in_movie_df - movies_in_user_df

print(f"Number of movies not rated by users: {len(unrated_movies)}")

Number of movies not rated by users: 16420


Combine these two dataframe objects in order to initalize our matrix. We assume that an "unrated" movie has a rating 0 for now.

In [40]:
merged_df = pd.merge(user_df, movie_df, on='Movie_ID')
rating_matrix = merged_df.pivot_table(index='User_ID', columns='Name', values='Rating')
rating_matrix = rating_matrix.fillna(0)
rating_matrix = rating_matrix.sort_index()

In [41]:
print(f"Pivot Table Size: {rating_matrix.shape[0]} rows, {rating_matrix.shape[1]} columns")
rating_matrix.head(10)

Pivot Table Size: 143458 rows, 1342 columns


Name,10,10 Things I Hate About You,101 Dalmatians II: Patch's London Adventure,11:14,13 Ghosts,18 Again,1984,2 Fast 2 Furious,200 Cigarettes,2010: The Year We Make Contact,...,Xena: Warrior Princess: Season 3,Xena: Warrior Princess: Series Finale,Y Tu Mama Tambien,Yellow Submarine,Yi Yi,Yojimbo,Young Black Stallion,Youngblood,Yu-Gi-Oh!: The Movie,Zorro
User_ID,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Running similiarity metrics on our rating matrix

Cosine similiarity: 
The cosine similarity between two vectors A and B is given by the formula: cos_similarity(A, B) = $\frac{A \cdot B}{\lVert \boldsymbol{A} \rVert \lVert \boldsymbol{B} \rVert}$

Since our dataset is large, for this technical demo, we will use the first 1,000 users to demonstrate the following similarity metrics. 

In [42]:
sample_rating_matrix = rating_matrix.head(1000)

In [43]:
def cosine_similarity(matrix):
    matrix_np = matrix.to_numpy()

    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((len(matrix), len(matrix)))

    for i in range(len(matrix)):
        for j in range(i, len(matrix)):  # Since matrix is symmetric, we only need to compute the upper half
            # Indices of non-zero elements for both users
            nz_indices_i = matrix_np[i, :] != 0
            nz_indices_j = matrix_np[j, :] != 0
            nz_indices_both = nz_indices_i & nz_indices_j

            # Vectors for computation
            vector_i = matrix_np[i, nz_indices_both]
            vector_j = matrix_np[j, nz_indices_both]

            # Skip if either vector is empty
            if len(vector_i) == 0 or len(vector_j) == 0:
                continue

            # Compute cosine similarity
            dot_product = np.dot(vector_i, vector_j)
            norm_i = np.linalg.norm(vector_i)
            norm_j = np.linalg.norm(vector_j)
            similarity = dot_product / (norm_i * norm_j)

            # Fill the similarity matrix
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

    return similarity_matrix

In [44]:
start = time.time()
def compute_for_user(user, matrix):
    cosine_sim_matrix = cosine_similarity(matrix)

    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)

    # Find the similarity values for User 6 and exclude user 
    similarity_with_user = cosine_sim_df[user]
    similarity_with_user = similarity_with_user.drop(user)

    # Predict ratings for user
    weighted_ratings = matrix.mul(similarity_with_user, axis=0).sum(axis=0) / similarity_with_user.sum()
    predicted_ratings = weighted_ratings

    # Exclude movies already rated by user
    unrated_movies = matrix.loc[user] == 0
    predicted_ratings_for_unrated_movies = predicted_ratings[unrated_movies]

    top_10_movies = predicted_ratings_for_unrated_movies.sort_values(ascending=False).head(10)
    return top_10_movies

top_10_movies = compute_for_user(6, sample_rating_matrix)
end = time.time()

print("Top 10 movies recommended for User 6:")
print(top_10_movies)
print("Execution time of the program is: ", end-start)


Top 10 movies recommended for User 6:
Name
American Beauty                          2.778856
Ghost                                    2.221978
Speed                                    1.930027
The Wizard of Oz: Collector's Edition    1.882315
Jaws                                     1.869414
Being John Malkovich                     1.860085
Patch Adams                              1.825309
Bend It Like Beckham                     1.821213
Sideways                                 1.786273
National Lampoon's Vacation              1.747065
dtype: float64
Execution time of the program is:  24.084681034088135


In [45]:
def cosine_similarity_optimized(matrix):
    # Convert to sparse matrix
    sparse_matrix = csr_matrix(matrix)

    # Compute the magnitude for each user vector
    magnitude = norm(sparse_matrix, axis=1)

    # Normalize each vector to unit norm
    unit_matrix = sparse_matrix.multiply(1 / magnitude.reshape(-1, 1))

    # Compute cosine similarity
    return unit_matrix.dot(unit_matrix.T).toarray()

In [46]:
start = time.time()
def compute_for_user(user, matrix):
    # Calculate the cosine similarity matrix
    cosine_sim_matrix = cosine_similarity_optimized(matrix)

    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)

    # Find the similarity values for user and exclude user 
    similarity_with_user = cosine_sim_df[user]
    similarity_with_user = similarity_with_user.drop(user)

    # Predict ratings for user
    weighted_ratings = matrix.mul(similarity_with_user, axis=0).sum(axis=0) / similarity_with_user.sum()
    predicted_ratings = weighted_ratings

    # Exclude movies already rated by user
    unrated_movies = matrix.loc[user] == 0
    predicted_ratings_for_unrated_movies = predicted_ratings[unrated_movies]

    top_10_movies = predicted_ratings_for_unrated_movies.sort_values(ascending=False).head(10)
    return top_10_movies

top_10_movies = compute_for_user(6, sample_rating_matrix)
end = time.time()

print("Top 10 movies recommended for User 6:")
print(top_10_movies)
print("Execution time of the program is:  ", end-start)



Top 10 movies recommended for User 6:
Name
American Beauty                          2.871258
Ghost                                    2.326168
Speed                                    2.084855
Jaws                                     2.014155
The Wizard of Oz: Collector's Edition    1.964019
Patch Adams                              1.918758
Being John Malkovich                     1.902401
Sideways                                 1.891653
National Lampoon's Vacation              1.890831
Bend It Like Beckham                     1.845176
dtype: float64
Execution time of the program is:   0.14936017990112305


As you can see from the output of my two cosine_similarity functions, the results are slightly different - the #1 suggested movie of American beautiy has a "score" of 2.77 in the first result and 2.88 in the second. The Wizard of Oz is the 4th recommended movie in the first implementation while it is 5th in the second implementation. 

Reasons for slight differences: 

a. Treatment of Zeros:

In the dense implementation (cosine_similarity), zeros are explicitly considered and filtered out for each pair of users being compared. This method calculates similarity based solely on the items that both users have rated.
The sparse implementation (cosine_similarity_optimized) inherently ignores zeros since they are not stored in the sparse matrix format. This method focuses on the non-zero ratings but might include zeros in the normalization step across entire vectors.

b. Normalization Process:

The first method normalizes vectors by considering only non-zero elements, effectively altering the vector length during normalization. This approach changes the basis of comparison to focus strictly on commonly rated items.
The sparse method normalizes each vector as a whole, including zeros. This can lead to a slightly different interpretation of user preferences, especially in rows with many zeros.

While there are technical differences in how the dense and sparse implementations of cosine similarity handle the data, especially regarding zeros and normalization, these differences are usually minor in the context of a recommendation system. The primary goal is to identify general patterns of user preferences, and both methods effectively contribute to this goal, with the sparse method offering significant computational advantages (as you can tell by the total time it took the program to run).